In [3]:
import gc

import numpy as np

"""
ToyNetwork 재분석
    화성~서울 네트워크
    집계시간 : 5분
    분석시간 1800~8400
"""
import pandas as pd

import os

# FIX 값 모음
###################################################################################################################

path = r"C:\VISSIM_Workspace\테스트\화성~서울"

start_interval = 1800
end_interval = 8400

weights = {
    "w1" : 1,
    "w2" : 1,
    "w3" : 1,
    "w4" : 1,
    "w5" : 1,
    "w6" : 1
}

vehicle_types = [100, 300, 630, 640, 650]

# 램프 간섭 영향률
###################################################################################################################
# 램프 전 본선 검지기(램프 간섭 영향률)
before_ramp = [70, 117, 124, 186, 215, 312, 342, 403, 412, 460]

# 램프 후 본선 검지기(램프 간섭 영향률)
after_ramp = [74, 121, 127, 190, 221, 317, 345, 406, 416, 465]

# 유입램프 검지기(램프 간섭 영향률)
input_ramp = [902, 904, 906, 908, 910]

# 유출램프 검지기(램프 간섭 영향률)
output_ramp = [901, 903, 905, 907, 909]

# 진출 정상성(진입)(진출 원활률)
enter_line = [23, 121, 190, 317, 406]

# 유입램프 바로 뒤 본선 검지기(진출 원활률)(램프 간섭 영향률)
input_main_ramp = [121, 190, 317, 406, 465]

# 유출램프 바로 앞 본선 검지기(진출 원활률)(램프 간섭 영향률)
output_main_ramp = [73, 126, 220, 344, 414]
###################################################################################################################

# 함수 모음
###################################################################################################################

# 속도 변화율
def speed_mean(original_df):
    copy_df = original_df.copy()

    # 램프 검지기 제외
    copy_df = copy_df[~copy_df["New_Measurement"].between(900, 910)]

    # TimeGroup, New_Measurement별 그룹화 및 속도 평균
    speed_mean_df = (
        copy_df.groupby(["TimeGroup", "New_Measurement"])
          .agg(V_mean=("v[km/h]", "mean"), V_count=("v[km/h]", "count"))
          .reset_index()
    )
    speed_mean_df["V_next"] = speed_mean_df.groupby("TimeGroup")["V_mean"].shift(-1)
    speed_mean_df["delta_V"] = (speed_mean_df["V_next"] - speed_mean_df["V_mean"]) / speed_mean_df["V_mean"]
    speed_mean_df["delta_V"] = speed_mean_df["delta_V"].fillna(0)

    return speed_mean_df

# 밀도 변화율
def density_mean(speed_df):
    copy_df = speed_df.copy()
    density_mean_df = copy_df.assign(K = copy_df["V_count"] * 12 / copy_df["V_mean"])
    density_mean_df["K_next"] = density_mean_df.groupby("TimeGroup")["K"].shift(-1)
    density_mean_df["delta_K"] = (density_mean_df["K_next"] - density_mean_df["K"]) / density_mean_df["K"]
    density_mean_df["delta_K"] = density_mean_df["delta_K"].fillna(0)

    return density_mean_df

# 중차량 혼입률
def heavy_rate(original_df):
    copy_df = original_df.copy()

    heavy_df = (
        copy_df[copy_df["Vehicle type"].isin([630, 640, 650])]
        .groupby(["TimeGroup", "New_Measurement"])
        .size()
        .reset_index(name="heavy_count")
    )
    # TimeGroup별 총 차량 갯수 집계
    total_df = (
        copy_df.groupby(["TimeGroup", "New_Measurement"])
        .size()
        .reset_index(name="total_count")
    )
    heavy_rate_df = pd.merge(
        heavy_df,
        total_df,
        on=["TimeGroup", "New_Measurement"],
        how="left"
    )
    heavy_rate_df["rate"] = heavy_rate_df["heavy_count"] / heavy_rate_df["total_count"]
    return heavy_rate_df


# 동적 포화도
def entry_saturation(original_df):
    copy_df = original_df.copy()

    # 실측용량 C(2차로 4400)
    max_capacity = 4400
    entry_saturation_df = (
        copy_df.groupby(["TimeGroup", "New_Measurement"])
        .size()
        .reset_index(name="entry_volume")  # 차량 수를 entry_volume이라는 컬럼명으로
    )

    # 단위가 대/시 이기 때문에 현재 5분집계 * 12
    entry_saturation_df["Phi_진입"] = entry_saturation_df["entry_volume"] * 12 / max_capacity
    return entry_saturation_df

# 램프 간섭 영향률
def rfr_rate(original_df):
    copy_df = original_df.copy()
    copy_df["TimeGroup"] = copy_df["TimeGroup"].astype(str)
    main_results=[]

    for i, (before, after) in enumerate(zip(before_ramp, after_ramp)):
        q_before = (copy_df[copy_df["New_Measurement"] == before] # 70, 117, 124, 186, 215, 312, 342, 403, 412, 460
                    .groupby("TimeGroup")
                    .size()
                    .reset_index(name="q_before"))

        q_after = (copy_df[copy_df["New_Measurement"] == after] # 74, 121, 127, 190, 221, 317, 345, 406, 416, 465
                   .groupby("TimeGroup")
                   .size()
                   .reset_index(name="q_after"))

        merged = q_before.merge(q_after, on="TimeGroup", how="outer").fillna(0)
        merged["Qm"] = (merged["q_before"] + merged["q_after"]) / 2
        main_results.append(merged)

    ramp_results = []
    for input_, output_ in zip(input_ramp, output_ramp):
        q_out = (copy_df[copy_df["New_Measurement"] == output_] # 901, 903, 905, 907, 909
                 .groupby("TimeGroup").size().reset_index(name="q_out"))
        ramp_results.append(q_out)

        q_in = (copy_df[copy_df["New_Measurement"] == input_] # 902, 904, 906, 908, 910
                .groupby("TimeGroup").size().reset_index(name="q_in"))
        ramp_results.append(q_in)



    input_queue = input_main_ramp.copy()
    output_queue = output_main_ramp.copy()
    rfr_list = []

    for i in range(min(len(main_results), len(ramp_results))):
        main_df = main_results[i]
        ramp_df = ramp_results[i]

        rfr_df = pd.merge(main_df, ramp_df, on="TimeGroup", how="outer").fillna(0)

        # 기본값 초기화
        rfr_df["IR_in"] = 0
        rfr_df["IR_out"] = 0

        # q_in 있을 때 (유입)
        if "q_in" in rfr_df.columns:
            rfr_df["IR_in"] = rfr_df["q_in"] / rfr_df["Qm"]
            if input_queue:  # 남은 게 있으면 하나 꺼냄
                current_input = input_queue.pop(0)
                rfr_df["New_Measurement"] = current_input

        # q_out 있을 때 (유출)
        if "q_out" in rfr_df.columns:
            rfr_df["IR_out"] = rfr_df["q_out"] / rfr_df["Qm"]
            if output_queue:
                current_output = output_queue.pop(0)
                rfr_df["New_Measurement"] = current_output

        rfr_df["RFR"] = rfr_df["IR_in"] + rfr_df["IR_out"]

        rfr_list.append(rfr_df)

    final_rfr_df = pd.concat(rfr_list, ignore_index=True)
    # -----------------------------
    # 특정 검지기에만 RFR 반영
    # -----------------------------
    target_measurements = input_main_ramp + output_main_ramp
    all_measurements = copy_df["New_Measurement"].unique()

    expanded_df_list = []

    base_rfr_df = final_rfr_df.copy()

    for m in all_measurements:
        if m in target_measurements:
            temp = base_rfr_df[base_rfr_df["New_Measurement"] == m].copy()
        else:
            temp = base_rfr_df[["TimeGroup"]].drop_duplicates().copy()
            temp["New_Measurement"] = m
            temp["RFR"] = 0

        expanded_df_list.append(temp)

    final_rfr_df = pd.concat(expanded_df_list, ignore_index=True)
    final_rfr_df = final_rfr_df.sort_values(by=["TimeGroup", "New_Measurement"]).reset_index(drop=True)
    final_rfr_df = final_rfr_df[["TimeGroup", "New_Measurement", "RFR"]]

    return final_rfr_df

# 진출 원활율- output_main_ramp
def output_normality(original_df):
    copy_df = original_df.copy()

    normality_list = []

    # 여러 진입/진출 쌍 처리
    for enter, exit_ in zip(enter_line, output_main_ramp):

        entry_df = copy_df[copy_df["New_Measurement"] == enter][["VehNo", "t(Entry)"]]
        exit_df  = copy_df[copy_df["New_Measurement"] == exit_][["VehNo", "t(Entry)"]]

        # 차량 번호별 최소 통과시각
        entry_first = (
            entry_df.groupby("VehNo")["t(Entry)"].min()
            .reset_index()
            .rename(columns={"t(Entry)": "t_entry"})
        )

        exit_first = (
            exit_df.groupby("VehNo")["t(Entry)"].min()
            .reset_index()
            .rename(columns={"t(Entry)": "t_exit"})
        )

        # 진입-진출 매칭 후 지연시간 계산
        merged = pd.merge(entry_first, exit_first, on="VehNo", how="inner")
        merged["delay_sec"] = merged["t_exit"] - merged["t_entry"]
        merged = merged[merged["delay_sec"] >= 0]  # 음수 제거

        # 중간값 기반 시간지연 bin 계산
        if len(merged) and np.isfinite(np.nanmedian(merged["delay_sec"])):
            lag_bins = int(round(np.nanmedian(merged["delay_sec"]) / 300))
        else:
            lag_bins = 0

        # 진입/진출 카운트 집계
        entry_count = (
            original_df[original_df["New_Measurement"] == enter]
            .groupby("TimeGroup")
            .size()
            .reset_index(name="Q_in")
        )

        exit_count = (
            original_df[original_df["New_Measurement"] == exit_]
            .groupby("TimeGroup")
            .size()
            .reset_index(name="Q_out")
        )

        # 병합 후 지연만큼 shift
        merged_counts = pd.merge(entry_count, exit_count, on="TimeGroup", how="left")
        merged_counts["Q_out_shift"] = merged_counts["Q_out"].shift(-lag_bins)

        # F(outrate)
        merged_counts["F(outrate)"] = (merged_counts["Q_out_shift"] / merged_counts["Q_in"]).fillna(0)
        merged_counts["New_Measurement"] = exit_  # 진출 지점에 부여

        normality_list.append(merged_counts)

    # 모든 진출 램프 결과 병합
    final_df = pd.concat(normality_list, ignore_index=True)


    # 전체 검지기 확장
    all_measurements = copy_df["New_Measurement"].unique()
    expanded_list = []

    for m in all_measurements:
        if m in output_main_ramp:
            temp = final_df[final_df["New_Measurement"] == m].copy()
        else:
            temp = final_df[["TimeGroup"]].drop_duplicates().copy()
            temp["New_Measurement"] = m
            temp["F(outrate)"] = 0
        expanded_list.append(temp)

    final_df = pd.concat(expanded_list, ignore_index=True)
    final_df = final_df.sort_values(by=["TimeGroup", "New_Measurement"]).reset_index(drop=True)

    return final_df


def calculate_stvm(speed_df, density_df, heavy_df, entry_saturation_df, rfr_df, normality_df):

    # TimeGroup 기준으로  Merge
    merged_df = (
        speed_df[["TimeGroup", "New_Measurement", "delta_V"]]
        .merge(density_df[["TimeGroup", "New_Measurement", "delta_K"]], on=["TimeGroup", "New_Measurement"])
        .merge(heavy_df[["TimeGroup", "New_Measurement", "rate"]], on=["TimeGroup", "New_Measurement"])
        .merge(entry_saturation_df[["TimeGroup", "New_Measurement", "Phi_진입"]], on=["TimeGroup", "New_Measurement"])
        .merge(rfr_df[["TimeGroup", "New_Measurement", "RFR"]], on=["TimeGroup", "New_Measurement"])
        .merge(normality_df[["TimeGroup", "New_Measurement", "F(outrate)"]], on=["TimeGroup", "New_Measurement"])
    )

    merged_df["STVM"] = (
        weights["w1"] * merged_df["delta_V"] +
        weights["w2"] * merged_df["delta_K"] +
        weights["w3"] * merged_df["rate"] +
        weights["w4"] * merged_df["Phi_진입"] +
        weights["w5"] * merged_df["RFR"] +
        weights["w6"] * merged_df["F(outrate)"]
    )
    merged_df = modify_frame(merged_df)
    return merged_df

def calc_z(df):
    if df.empty:
        return df
    mean_stvm = df["STVM"].mean(skipna=True)
    std_stvm = df["STVM"].std(skipna=True)
    df["Z-Score"] = (df["STVM"] - mean_stvm) / std_stvm
    z_max = df["Z-Score"].max()
    z_min = df["Z-Score"].min()
    df["환산점수"] = df["Z-Score"].apply(lambda z: z_to_score(z, z_min, z_max))
    return df

def calculate_z_score(stvm_df):
    copy_df = stvm_df.copy()


    # 구간 나누기
    group1 = copy_df[copy_df["New_Measurement"].between(1, 265)].copy()
    group2 = copy_df[copy_df["New_Measurement"].between(266, 530)].copy()

    group1 = calc_z(group1)
    group2 = calc_z(group2)

    # 두 구간 합치기
    merged = pd.concat([group1, group2], ignore_index=True)
    merged = merged.sort_values(by=["TimeGroup", "New_Measurement"])
    stvm_df = pd.pivot(merged, index="TimeGroup", columns= "New_Measurement", values="환산점수")
    return stvm_df

def modify_frame(df):
    modify_df = df.copy()
    modify_df["StartTime"] = modify_df["TimeGroup"].str.split("~").str[0].astype(int)
    modify_df = modify_df[(modify_df["StartTime"] >=1800) &(modify_df["StartTime"] < 8400)]
    modify_df = modify_df[~modify_df["New_Measurement"].isin([266, 531, 901, 902, 903, 904, 905, 906, 907, 908, 909, 910])]
    return modify_df


def variable_timegroup_avg(stvm_df):
    copy_df = stvm_df.copy()
    variable_time_df = copy_df.groupby("TimeGroup")[["delta_V", "delta_K", "rate", "Phi_진입", "RFR", "F(outrate)"]].mean()
    return variable_time_df

def variable_total_avg(variable_df):
    variable_total_df = pd.DataFrame([variable_df.mean(numeric_only=True)])
    return variable_total_df

def speed_density_avg(density_df):
    copy_df = density_df.copy()
    avg_df = modify_frame(copy_df)
    avg_df = pd.DataFrame([avg_df.mean(numeric_only=True)])
    avg_df = avg_df[["V_mean", "K"]]
    return avg_df

def pivot_table(df, value, preprocess=None):
    copy_df = df.copy()
    if preprocess :
        copy_df = preprocess(copy_df)
    return copy_df.pivot(index="TimeGroup", columns="New_Measurement", values=value)

def weighted_avg_speed(original_df):
    copy_df = original_df.copy()
    # TimeGroup, New_Measurement별 그룹화 및 속도 평균
    speed_mean_df = (
        copy_df.groupby(["TimeGroup", "New_Measurement", "Vehicle type"])
          .agg(V_mean=("v[km/h]", "mean"), V_count=("v[km/h]", "count"))
          .reset_index()
    )
    speed_mean_df["std_group"] = speed_mean_df.groupby(["TimeGroup", "New_Measurement"])["V_mean"].transform(lambda s: s.std(ddof=0))
    speed_mean_df["cv"] = speed_mean_df["std_group"] / speed_mean_df["V_mean"]
    speed_mean_df["w"] = 1 / speed_mean_df["cv"]
    speed_mean_df["w*v"] = speed_mean_df["w"] * speed_mean_df["V_mean"]

    weighted_result = (
        speed_mean_df.groupby(["TimeGroup","New_Measurement"])
          .apply(lambda g: g["w*v"].sum() / g["w"].sum())
          .reset_index(name="Weighted_Avg_Speed")
    )

    return weighted_result

def save_to_excel(excel_df, folder_path, file_name, i):
        excel_folder_path = os.path.join(folder_path, file_name)
        os.makedirs(excel_folder_path, exist_ok=True)
        excel_file_name = f"{file_name}_{i+1}.xlsx"
        excel_file_path = os.path.join(excel_folder_path, excel_file_name)
        excel_df.to_excel(excel_file_path, index=True)
        print(f"{excel_file_name} 생성 완료")

def z_to_score(z, z_min, z_max):
    if 1.645 <= z <= z_max:
        return 50 + ((95 + 5 * ((z - 1.645) / (z_max - 1.645))) * 0.5)
    elif 1.282 <= z < 1.645:
        return 50 + ((90 + 5 * ((z - 1.282) / (1.645 - 1.282))) * 0.5)
    elif 1.038 <= z < 1.282:
        return 50 + ((85 + 5 * ((z - 1.038) / (1.282 - 1.038))) * 0.5)
    elif 0.842 <= z < 1.038:
        return 50 + ((80 + 5 * ((z - 0.842) / (1.038 - 0.842))) * 0.5)
    elif 0.676 <= z < 0.842:
        return 50 + ((75 + 5 * ((z - 0.676) / (0.842 - 0.676))) * 0.5)
    elif 0.526 <= z < 0.676:
        return 50 + ((70 + 5 * ((z - 0.526) / (0.676 - 0.526))) * 0.5)
    elif 0.387 <= z < 0.526:
        return 50 + ((65 + 5 * ((z - 0.387) / (0.526 - 0.387))) * 0.5)
    elif 0.255 <= z < 0.387:
        return 50 + ((60 + 5 * ((z - 0.255) / (0.387 - 0.255))) * 0.5)
    elif -0.255 <= z < 0.255:
        return 50 + ((40 + 5 * ((z + 0.255) / (0.255 + 0.255))) * 0.5)
    elif -0.387 <= z < -0.255:
        return 50 + ((35 + 5 * ((z + 0.387) / (-0.255 + 0.387))) * 0.5)
    elif -0.526 <= z < -0.387:
        return 50 + ((30 + 5 * ((z + 0.526) / (-0.387 + 0.526))) * 0.5)
    elif -0.676 <= z < -0.526:
        return 50 + ((25 + 5 * ((z + 0.676) / (-0.676 + 0.842))) * 0.5)
    elif -0.842 <= z < -0.676:
        return 50 + ((20 + 5 * ((z + 0.842) / (-0.676 + 0.842))) * 0.5)
    elif -1.038 <= z < -0.842:
        return 50 + ((15 + 5 * ((z + 1.038) / (-0.842 + 1.038))) * 0.5)
    elif -1.282 <= z < -1.038:
        return 50 + ((10 + 5 * ((z + 1.282) / (-1.038 + 1.282))) * 0.5)
    elif -1.645 <= z < -1.282:
        return 50 + ((5 + 5 * ((z + 1.645) / (-1.282 + 1.645))) * 0.5)
    elif z_min <= z < -1.645:
        return 50 + ((0 + 5 * ((z + z_min) / (-1.645 + z_min))) * 0.5)
    else:
        return np.nan
###################################################################################################################

folder_path = path
mer_list = [file for file in os.listdir(folder_path) if file.endswith(".mer")]

grouped_df = pd.DataFrame()
result_df = pd.DataFrame()

for i in range(len(mer_list)):
    mer_file = mer_list[i]
    print("작업파일 : ", mer_file)
    with open(os.path.join(folder_path, mer_file), "r", encoding="utf-8", errors="ignore") as file:
            lines = file.readlines()
            # 데이터가 시작하는 인덱스 찾기
            data_start_idx = None

            for j, line in enumerate(lines):
                if "Measurem." in line:  # 컬럼명이 포함된 행 찾기
                    data_start_idx = j
                    break

            # 데이터프레임 생성
            if data_start_idx is not None:

                # 컬럼명 추출 및 공백 제거
                columns = [col.strip() for col in lines[data_start_idx].strip().split(";")]

                # 데이터 부분 추출 및 가공
                data_lines = lines[data_start_idx + 1:]  # 컬럼명 제외, 데이터 부분
                data = [line.strip().split(";") for line in data_lines if line.strip()]

                # 데이터프레임 생성
                df = pd.DataFrame(data, columns=columns)

                # 컬럼 내부 데이터 정수형 변환
                df = df.apply(pd.to_numeric, errors="coerce")

                original_df = df[(df["t(Entry)"] != -1.00)].reset_index(drop=True)


                #불필요 컬럼 제거
                original_df.drop(columns=["b[m/s2]", "tQueue", "Occ", "Pers"], inplace=True, errors="ignore")

                original_df["New_Measurement"] = original_df["Measurem."] % 1000

                bins = np.arange(start_interval, end_interval+1, 300)
                labels = [f"{start}~{start+300}" for start in bins[:-1]]  # 구간 라벨링

                # 구간 나누기 및 컬럼 추가
                original_df["TimeGroup"] = pd.cut(original_df["t(Entry)"], bins=bins, labels=labels, right=False)

                # 가감속 차로 부분 검지기 제거
                original_df = original_df[~original_df["Measurem."].between(30000, 39999)]

                # 속도변화율
                speed_df = speed_mean(original_df)

                # 밀도변화율
                density_df = density_mean(speed_df)

                # 중차량 혼입률
                heavy_df = heavy_rate(original_df)

                # 동적 포화도
                entry_saturation_df = entry_saturation(original_df)

                # 램프 간섭 영향률
                rfr_df = rfr_rate(original_df)

                # 진출 원활율
                normality_df = output_normality(original_df)

                # STVM 계산
                stvm_df = calculate_stvm(speed_df, density_df, heavy_df, entry_saturation_df, rfr_df, normality_df)
                #display("stvm_df : ", stvm_df)
                #save_to_excel(stvm_df, folder_path, "STVM", i)

                # Z-Score 계산
                z_score_df = calculate_z_score(stvm_df)
                #save_to_excel(z_score_df, folder_path, "환산점수", i)

                # STVM 피봇
                stvm_pivot_df = pivot_table(stvm_df, "STVM")
                display("stvm_pivot_df : ", stvm_pivot_df)
                #save_to_excel(stvm_pivot_df, folder_path, "STVM_PIVOT", i)

                # 속도값 피봇
                #speed_pivot_df = pivot_table(speed_df, "V_mean", preprocess=modify_frame)
                #display("speed_pivot_df : ", speed_pivot_df)
                #save_to_excel(speed_pivot_df, folder_path, "속도값", i)

                # 메모리 정리
                #del df, original_df, speed_df, density_df, heavy_df, entry_saturation_df, rfr_df, normality_df, stvm_df, z_score_df
                gc.collect()


작업파일 :  화성~서울_250908_001.mer
STVM_1.xlsx 생성 완료


'stvm_pivot_df : '

New_Measurement,1,2,3,4,5,6,7,8,9,10,...,521,522,523,524,525,526,527,528,529,530
TimeGroup,,,,,,,,,,,,,,,,,,,,,
1800~2100,0.591409,0.611860,0.589083,0.607709,0.595760,0.595759,0.609060,0.611941,0.606827,0.626461,...,0.770767,0.775408,0.792838,0.761383,0.770292,0.751160,0.764784,0.772358,0.807411,0.837987
2100~2400,0.585317,0.585161,0.606921,0.600865,0.602812,0.614471,0.600976,0.581622,0.596998,0.563629,...,0.746510,0.719965,0.726920,0.761574,0.769447,0.784431,0.775008,0.743089,0.725717,0.689625
2400~2700,0.624915,0.622483,0.606754,0.614756,0.602918,0.601032,0.599161,0.615387,0.612945,0.618915,...,0.640444,0.665863,0.667580,0.679640,0.679892,0.685271,0.687733,0.722509,0.723299,0.737363
2700~3000,0.720178,0.720167,0.723906,0.723906,0.736117,0.733991,0.746148,0.730119,0.723252,0.735629,...,0.728669,0.723790,0.716470,0.721634,0.709331,0.693629,0.690323,0.681511,0.654161,0.650233
3000~3300,0.587431,0.593783,0.595457,0.584566,0.568279,0.595381,0.579290,0.597033,0.606723,0.574461,...,0.667000,0.682796,0.694834,0.674415,0.700549,0.718722,0.723731,0.695491,0.704782,0.713131
3300~3600,0.655872,0.640886,0.642760,0.654016,0.652023,0.640902,0.642764,0.640930,0.622024,0.651287,...,0.685920,0.684112,0.646183,0.644205,0.616250,0.624434,0.612652,0.624042,0.629804,0.657297
3600~3900,0.572013,0.572029,0.578290,0.567629,0.578309,0.586411,0.583798,0.585667,0.599714,0.603316,...,0.541719,0.508923,0.561354,0.565326,0.573905,0.565478,0.571731,0.605623,0.623437,0.604068
3900~4200,0.673953,0.677721,0.677695,0.683214,0.684893,0.670268,0.666663,0.652042,0.635228,0.604879,...,0.737007,0.762689,0.760700,0.767000,0.761523,0.744580,0.739007,0.725399,0.745408,0.753737
4200~4500,0.581008,0.596983,0.581027,0.584047,0.575839,0.580679,0.593752,0.603402,0.613073,0.643028,...,0.708175,0.696125,0.696202,0.687654,0.691804,0.704631,0.711554,0.714893,0.695286,0.695940


STVM_PIVOT_1.xlsx 생성 완료
작업파일 :  화성~서울_250908_002.mer
STVM_2.xlsx 생성 완료


'stvm_pivot_df : '

...

New_Measurement,1,2,3,4,5,6,7,8,9,10,...,521,522,523,524,525,526,527,528,529,530
TimeGroup,,,,,,,,,,,,,,,,,,,,,
1800~2100,1.223310,1.212195,1.144862,1.101721,1.022613,0.994125,1.007364,0.993532,1.008581,1.001993,...,1.023323,1.052013,1.047389,1.051430,1.044726,1.049037,1.038202,1.007188,1.000646,0.985148
2100~2400,1.036803,1.072067,1.060673,1.094774,1.157544,1.176045,1.155783,1.171939,1.167151,1.181160,...,1.051818,1.010839,0.998186,0.975060,0.959505,0.926860,0.906526,0.923164,0.910228,0.919255
2400~2700,1.271153,1.238758,1.221882,1.212223,1.204123,1.215646,1.226222,1.213485,1.212064,1.220172,...,1.073241,1.076967,1.081555,1.114434,1.128743,1.143930,1.167779,1.159293,1.160724,1.159856
2700~3000,1.225921,1.223510,1.195245,1.204029,1.206316,1.189435,1.192176,1.195031,1.209524,1.198875,...,0.929908,0.948755,0.954299,0.911820,0.923389,0.930212,0.924759,0.952850,0.984096,0.992069
3000~3300,1.277894,1.257823,1.226659,1.212358,1.208960,1.218301,1.217547,1.209580,1.195188,1.198689,...,0.998307,0.999356,0.979541,1.011152,0.976475,0.962487,0.975921,0.962357,0.929205,0.917765
3300~3600,1.250185,1.205870,1.151542,1.103127,1.107094,1.107164,1.102862,1.106883,1.109085,1.099008,...,1.052418,1.035254,1.047792,1.023865,1.026128,1.033992,1.008642,1.019707,1.055869,1.039278
3600~3900,1.146040,1.142281,1.160380,1.191550,1.191184,1.205206,1.195783,1.202294,1.204891,1.201179,...,1.094077,1.093091,1.089229,1.090634,1.105954,1.101875,1.111822,1.092925,1.085759,1.076022
3900~4200,1.257118,1.241201,1.206909,1.204187,1.212941,1.188490,1.199075,1.200534,1.200412,1.208414,...,0.991540,0.983655,0.993093,0.990475,0.976542,1.001062,0.995508,1.002609,0.991741,1.025230
4200~4500,1.250414,1.236015,1.216430,1.202939,1.152132,1.116225,1.047843,0.998793,1.005373,1.001813,...,1.138651,1.151913,1.149393,1.151396,1.149257,1.131536,1.129658,1.148930,1.144572,1.137606


STVM_PIVOT_3.xlsx 생성 완료
작업파일 :  화성~서울_250908_004.mer
STVM_4.xlsx 생성 완료


'stvm_pivot_df : '

New_Measurement,1,2,3,4,5,6,7,8,9,10,...,521,522,523,524,525,526,527,528,529,530
TimeGroup,,,,,,,,,,,,,,,,,,,,,
1800~2100,1.143172,1.167468,1.097614,1.006634,0.957565,0.904867,0.871753,0.889631,0.883517,0.895660,...,0.997449,0.987311,0.980620,0.966218,0.941030,0.926062,0.908662,0.933968,0.934983,0.924989
2100~2400,0.920072,0.947553,0.970867,1.024220,1.061183,1.095668,1.118577,1.110556,1.105445,1.103221,...,0.957728,0.968395,0.972906,0.951372,0.957914,0.959659,0.985771,0.962416,0.946692,0.947485
2400~2700,1.136273,1.087599,1.069222,1.068819,1.064500,1.056610,1.045332,1.048090,1.054782,1.056984,...,1.027602,1.033544,1.010577,1.039494,1.051588,1.035097,1.017121,1.046208,1.042654,1.051533
2700~3000,1.179058,1.196235,1.167425,1.149108,1.149845,1.150211,1.160840,1.157162,1.159075,1.162662,...,0.914852,0.934210,0.980722,0.985112,0.984889,0.986653,0.990379,0.977555,0.983610,0.964790
3000~3300,1.018736,1.031806,1.000167,0.994062,0.983983,0.987082,0.986676,0.988299,0.982716,0.984977,...,0.940185,0.907891,0.878209,0.857347,0.846550,0.883298,0.875936,0.873319,0.883564,0.873653
3300~3600,1.162975,1.182908,1.169083,1.171340,1.167880,1.172259,1.166007,1.169565,1.166474,1.153095,...,0.804609,0.824590,0.819969,0.846466,0.848922,0.850781,0.878123,0.896307,0.908413,0.944602
3600~3900,1.159201,1.166373,1.127624,1.114136,1.122499,1.110728,1.111138,1.107857,1.118162,1.127056,...,1.041836,1.038624,1.052560,1.031588,1.035071,1.012733,0.988188,0.978305,0.943001,0.916213
3900~4200,1.152321,1.146851,1.114786,1.109284,1.107012,1.115111,1.104895,1.107605,1.110701,1.104417,...,0.874835,0.883498,0.892246,0.921993,0.904851,0.932940,0.956751,0.949360,0.981935,0.977927
4200~4500,0.895847,0.853493,0.806288,0.805080,0.780706,0.791010,0.817749,0.793499,0.752005,0.668294,...,1.032427,1.019123,0.994565,0.962520,0.951467,0.928429,0.900172,0.911557,0.892564,0.887177


STVM_PIVOT_4.xlsx 생성 완료
